In [235]:
import numpy as np 
import pandas as pd
import requests 

from bs4 import BeautifulSoup

In [236]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_cities_by_average_temperature')
soup = BeautifulSoup(r.text,'html.parser')

In [237]:
r.text[0:500]

'\n<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of cities by average temperature - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"d7c34b01-06fe-42c'

In [238]:
continent = soup.find_all('span', attrs={'class':'mw-headline'})
continent

[<span class="mw-headline" id="Africa">Africa</span>,
 <span class="mw-headline" id="Asia">Asia</span>,
 <span class="mw-headline" id="Europe">Europe</span>,
 <span class="mw-headline" id="North_America">North America</span>,
 <span class="mw-headline" id="Oceania">Oceania</span>,
 <span class="mw-headline" id="South_America">South America</span>,
 <span class="mw-headline" id="See_also">See also</span>,
 <span class="mw-headline" id="References">References</span>]

In [302]:
results = soup.find_all('td')
len(results)

6784

In [250]:
temperature = list()
count = 0
c = 0
temp = list()
for idx in range(len(results)):
    try:
        if count == 0:
            country = results[idx].find('a')['title']
            count += 1
            temp.append(country)
        else: 
            if idx == 3457: # There is dirty data in here -_- monaco didn't have a city -_-
                city = results[idx].contents[0] 
                count = 0
                temp.append(city)
                
            else:
                city = results[idx].find('a')['title']
                count = 0
                temp.append(city)
    except:
        temp.append(results[idx].contents[0])
        c+=1
    if c == 14:
        temperature.append(tuple(temp))
        temp = list()
        c = 0

In [251]:
columns = ['Country','City','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec','Avg_Year', 'other']

In [252]:
world_temp_2020 = pd.DataFrame(temperature,columns = columns)
world_temp_2020.drop(columns = 'other',inplace=True)

In [253]:
world_temp_2020.tail()

,Country,City,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Avg_Year
419,Peru,Lima,22.1,22.7,22.2,20.6,18.8,17.5,16.7,16.2,16.4,17.3,18.7,20.7,19.2
420,Peru,Piura,26.8,27.8,27.8,26.6,24.5,22.9,21.8,21.9,22.3,22.9,23.6,25.2,24.5
421,Suriname,Paramaribo,26.0,26.0,26.2,26.7,26.5,26.5,26.7,27.4,27.8,27.9,27.3,26.4,26.8
422,Uruguay,Montevideo,23.0,22.5,20.6,17.2,14.0,11.1,10.9,11.7,13.4,16.0,18.6,21.3,16.7
423,Venezuela,Caracas,20.2,20.8,21.7,22.7,23.2,22.6,22.2,22.3,22.7,22.4,22.0,20.7,22.0


### Change the data type from beautifulsoup object into float

In [327]:
num_col = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul',
       'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Avg_Year']

(848, 1)

In [329]:
Month = pd.DataFrame()
for col in num_col:
    month = list()
    for i in range(len(world_temp_2020)):
        try:
            month.append(float(world_temp_2020.iloc[i,:][col]))
        except:
            month.append(float(world_temp_2020.iloc[i,:][col][1:])*-1)
    Month[col] = month


In [332]:
for col in num_col:
    world_temp_2020[col] = Month[col]

In [333]:
world_temp_2020.head()

,Country,City,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Avg_Year
0,Algeria,Algiers,11.2,11.9,12.8,14.7,17.7,21.3,24.6,25.2,23.2,19.4,15.2,12.1,17.4
1,Algeria,Tamanrasset,12.8,15.0,18.1,22.2,26.1,28.9,28.7,28.2,26.5,22.4,17.3,13.9,21.7
2,Algeria,Reggane,16.0,18.2,23.1,27.9,32.2,36.4,39.8,38.4,35.5,29.2,22.0,17.8,28.3
3,Angola,Luanda,26.7,28.5,28.6,28.2,27.0,23.9,22.1,22.1,23.5,25.2,26.7,26.9,25.8
4,Benin,Cotonou,27.3,28.5,28.9,28.6,27.8,26.5,25.8,25.6,26.0,26.7,27.6,27.3,27.2


### Adding Continent Column

In [334]:
Continent = list()

In [335]:
for i in range(len(world_temp_2020)):
    if i <= 108:
        Continent.append('Africa')
    elif i > 108 and i<= 183:
        Continent.append('Asia')
    elif i > 183 and i <= 245:
        Continent.append('Europe')
    elif i > 245 and i <= 343:
        Continent.append('North America')
    elif i > 343 and i<=369:
        Continent.append('Oceania')
    elif i > 369:
        Continent.append('South America')

In [336]:
world_temp_2020['Continent'] = Continent

In [337]:
world_temp_2020.head()

,Country,City,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Avg_Year,Continent
0,Algeria,Algiers,11.2,11.9,12.8,14.7,17.7,21.3,24.6,25.2,23.2,19.4,15.2,12.1,17.4,Africa
1,Algeria,Tamanrasset,12.8,15.0,18.1,22.2,26.1,28.9,28.7,28.2,26.5,22.4,17.3,13.9,21.7,Africa
2,Algeria,Reggane,16.0,18.2,23.1,27.9,32.2,36.4,39.8,38.4,35.5,29.2,22.0,17.8,28.3,Africa
3,Angola,Luanda,26.7,28.5,28.6,28.2,27.0,23.9,22.1,22.1,23.5,25.2,26.7,26.9,25.8,Africa
4,Benin,Cotonou,27.3,28.5,28.9,28.6,27.8,26.5,25.8,25.6,26.0,26.7,27.6,27.3,27.2,Africa


In [338]:
world_temp_2020.tail()

,Country,City,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Avg_Year,Continent
419,Peru,Lima,22.1,22.7,22.2,20.6,18.8,17.5,16.7,16.2,16.4,17.3,18.7,20.7,19.2,South America
420,Peru,Piura,26.8,27.8,27.8,26.6,24.5,22.9,21.8,21.9,22.3,22.9,23.6,25.2,24.5,South America
421,Suriname,Paramaribo,26.0,26.0,26.2,26.7,26.5,26.5,26.7,27.4,27.8,27.9,27.3,26.4,26.8,South America
422,Uruguay,Montevideo,23.0,22.5,20.6,17.2,14.0,11.1,10.9,11.7,13.4,16.0,18.6,21.3,16.7,South America
423,Venezuela,Caracas,20.2,20.8,21.7,22.7,23.2,22.6,22.2,22.3,22.7,22.4,22.0,20.7,22.0,South America


In [ ]:
world_temp_2020.info()

In [ ]:
world_temp_2020.to_csv('Avg_World_Temp_2020.csv')